In [1]:
%%capture
!pip install transformers[sentencepiece]
import transformers

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"    
import tensorflow as tf

Processing the data.

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# PATH_TO_FOLDER = "drive/MyDrive/Name2Demographics/"
PATH_TO_FOLDER = "../../../"
%cd $PATH_TO_FOLDER

/home/souvic/mounted/btp/vahini/Name2Demographics


In [5]:
import pandas as pd
import numpy as np

import sys  

sys.path.insert(0, 'Models/AIEEEDataState/PreProcessing/')
from aieee_caste import *

In [6]:
%%capture
!pip install datasets

In [7]:
caste_df = AIEEECasteStateFinalData()

/home/souvic/mounted/btp/vahini/Name2Demographics/Models/AIEEEDataState/PreProcessing/aieee_caste.py:251: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category'] = df['category'].replace('', np.NaN)


In [8]:
caste_df.head()

,Name,Caste
0,gauri shankar pandey 35,0.0
1,nelluri teja bharath 14,0.0
2,jerin t jerome 04,1.0
3,bhuri gurjar 16,1.0
4,abhishek deopa 27,0.0


In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("google/muril-base-cased", num_labels=2)

In [10]:
from sklearn.model_selection import train_test_split

X = list(caste_df["Name"])
y = list(map(int,caste_df["Caste"]))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, return_tensors="pt")
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, return_tensors="pt")

In [11]:
caste_df.groupby('Caste').count()/caste_df.shape[0] * 100

,Name
Caste,
0.0,58.172387
1.0,41.827613


In [12]:
import torch

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [13]:
from transformers import TrainingArguments
from transformers import Trainer

In [14]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import EarlyStoppingCallback


# Define Trainer parameters
def compute_metric(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
#     print('here')

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    g_recall = recall_score(y_true=labels, y_pred=pred, pos_label=0)
    g_precision = precision_score(y_true=labels, y_pred=pred, pos_label=0)
    g_f1 = f1_score(y_true=labels, y_pred=pred,pos_label=0)
    r_recall = recall_score(y_true=labels, y_pred=pred, pos_label=1)
    r_precision = precision_score(y_true=labels, y_pred=pred, pos_label=1)
    r_f1 = f1_score(y_true=labels, y_pred=pred,pos_label=1)
    return {"accuracy": accuracy, "general_precision": g_precision, "general_recall":g_recall, "general_f1": g_f1,
        "reserved_precision": r_precision, "reserved_recall": r_recall, "reserved_f1": r_f1 }

# Define Trainer
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=100,
    per_device_eval_batch_size=100,
    num_train_epochs=1,
    seed=42,
    load_best_model_at_end=True,
#     learning_rate=1e-03,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metric,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Train pre-trained model
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vahini10 (use `wandb login --relogin` to force relogin)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/tmp/ipykernel_11519/3554856749.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,General Precision,General Recall,General F1,Reserved Precision,Reserved Recall,Reserved F1
500,0.610000,0.540672,0.707905,0.707146,0.849359,0.771756,0.709662,0.511428,0.594454
1000,0.534500,0.510539,0.728996,0.755037,0.790283,0.772258,0.688511,0.643869,0.665442


/tmp/ipykernel_11519/3554856749.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_11519/3554856749.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
metrics=trainer.evaluate()
print(metrics)

/tmp/ipykernel_11519/3554856749.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.4984588027000427, 'eval_accuracy': 0.7379236558610851, 'eval_general_precision': 0.7726423216648117, 'eval_general_recall': 0.7782478108211007, 'eval_general_f1': 0.7754349360899483, 'eval_reserved_precision': 0.6888559133402582, 'eval_reserved_recall': 0.6819143310417768, 'eval_reserved_f1': 0.6853675461001285, 'eval_runtime': 120.5235, 'eval_samples_per_second': 674.69, 'eval_steps_per_second': 6.754, 'epoch': 1.0}


In [16]:
metrics

{'eval_loss': 0.4984588027000427,
 'eval_accuracy': 0.7379236558610851,
 'eval_general_precision': 0.7726423216648117,
 'eval_general_recall': 0.7782478108211007,
 'eval_general_f1': 0.7754349360899483,
 'eval_reserved_precision': 0.6888559133402582,
 'eval_reserved_recall': 0.6819143310417768,
 'eval_reserved_f1': 0.6853675461001285,
 'eval_runtime': 120.5235,
 'eval_samples_per_second': 674.69,
 'eval_steps_per_second': 6.754,
 'epoch': 1.0}

In [17]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Fri Mar 18 13:20:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/

## Save the Model

In [16]:
torch.save(model.state_dict(), "Models/AIEEEData/MurilBERT/SavedModel/MurilBERT")

In [18]:
model.save_pretrained("Models/AIEEEDataState/MurilBERTWithState/SavedModel/MurilBERT")